In [1]:
import pandas as pd
import pymysql
import json
host ="59.15.106.157" 
port=9723 
username="lol" 
password ="1234"
conn = pymysql.connect(host=host, user=username, passwd=password, port=port, use_unicode=True, charset='utf8', autocommit=True, cursorclass=pymysql.cursors.DictCursor, db='lol')
cursor = conn.cursor()
query = "SELECT match_data FROM lol.match_details LIMIT 100;"
cursor.execute(query)
rows=cursor.fetchall()


In [2]:
required_columns = ['assistMePings', 'lane', 'championId', 'championName',
       'individualPosition', 'totalMinionsKilled', 'summonerNmae', 'win',
       'kills', 'deaths', 'assists', 'summonerId', 'summonerLevel', 'summonerName',
       'role', 'puuid']

In [3]:
dfs = []
for row in rows:
    match_data = json.loads(row['match_data'])
    
    for participant in  match_data['info']['participants']:
        match_row = {}
        match_row['match_id'] = match_data['metadata']['matchId']
        match_row['play_time'] = round(match_data['info']['gameDuration'] / 60, 2)
        for key in required_columns:
            if key in participant:
                match_row[key] = participant[key]
        dfs.append(pd.DataFrame([match_row])) 
match_df = pd.concat(dfs, ignore_index=True)
match_df['minionsKillperMin'] = match_df['totalMinionsKilled'] / match_df['play_time']
match_df['kda'] = match_df.apply(calculate_kda, axis=1)

In [18]:
def calculate_kda(row):
    k, d, a = row['kills'], row['deaths'], row['assists']
    if d == 0:
        return k + a
    return (k + a) / d

# 1. 탑 라인에서 assistMePings가 상위 10%인 경우

In [12]:
# Given match df, return top 10% assistMePings in top lane
def top_hanamja(df):
    top_df = df[df.lane.eq("TOP") & df.individualPosition.eq("TOP")]
    assistMePings_top_10 = top_df.assistMePings.quantile(0.9)
    return top_df[top_df.assistMePings > assistMePings_top_10].sort_values('assistMePings', ascending=False)
    

In [10]:
top_match_df = match_df[match_df.lane.eq("TOP") & match_df.individualPosition.eq("TOP")]
assistMePings_top_10 = top_match_df.assistMePings.quantile(0.9)
top_match_df[top_match_df.assistMePings > assistMePings_top_10].sort_values('assistMePings', ascending=False)

,match_id,play_time,assistMePings,lane,championId,championName,individualPosition,totalMinionsKilled,win,kills,deaths,assists,summonerId,summonerLevel,summonerName,role,puuid,minionsKillperMin,kda
385,KR_6435402918,33.10,17,TOP,133,Quinn,TOP,176,False,12,5,8,DI3h5fCcccn4BhDJ155y0miTfj8VBP9wQv90ENfBysbIIb0,273,엄 홍 섭,SOLO,W2z4GmdjQM4D233myjc-71oEn1JKBArj8y2rilyJc39JvA...,5.317221,4.000000
545,KR_6436432607,37.83,14,TOP,54,Malphite,TOP,251,True,7,7,22,_mopZZDTMuf5ShUqr5ak-rf5x3NOq-0EEFV70eTsRsb4I6c,105,가재맨2,SOLO,XFrciofmzlmyr9OoXfdOJGvwRqE9JackJBeFZ4gdw9J5lv...,6.634946,4.142857
435,KR_6435682929,20.33,12,TOP,86,Garen,TOP,125,True,3,1,0,Xhd8utKkly2ZP8u-l0sWuB0TdmjIuC4lzu8PEzFvXN1cJL0,599,미분반대는적분,SOLO,hcFGD0o48wMbQoKiUSB_DYCoD4AFPF5CB89g5-DC0g2Cnw...,6.148549,3.000000
950,KR_6438067077,42.20,10,TOP,516,Ornn,TOP,249,False,6,6,11,-nDwxnMp4HPWBkIswuMCW9nPro1OQjsdcBuW38Wt8gGOvUc,328,강서구현빈,SOLO,r_FA6K6MipDVdCIXQJHCXXR4kPcxSqXuo1U5J-TYxyfTk4...,5.900474,2.833333
587,KR_6436538119,36.23,10,TOP,113,Sejuani,TOP,167,False,6,7,6,FVNm0x4VkBKTva94l1oh1RF-fKAxXS4pxAjpnRlmT2TyR48,412,럼킹블,SOLO,OBxjok_YaW-1CQMgegYbR0bOx9aUr5J3vVN-h3jrGtXS6F...,4.609440,1.714286
445,KR_6435702020,33.90,10,TOP,114,Fiora,TOP,249,True,9,5,6,wzxVhcSXHB-wCigRykpxAX5c_oaQudi6uIxH5d938fOE5w,192,부천영도,SOLO,DHzQIuTgc86-qRSzYQekKoH6YZLTFnUzHSLLeMhBvPiPze...,7.345133,3.000000
225,KR_6434466840,24.75,8,TOP,107,Rengar,TOP,157,False,10,5,0,Ps1IlONlG0ODiZg4OuiVzU5ajgV_Th8CJmPifGkdvGt1fo8,294,사 힐,SOLO,jAgZEI6cVzhuTJZ3bDw11UMsef0sgnDCr7wgRW4URSK4BK...,6.343434,2.000000
690,KR_6436853204,37.10,7,TOP,24,Jax,TOP,147,False,0,9,4,UBW7PhBqJEiSYsfIq3z38Eh_xFw0zB92O0e4J6oHW57WTA,168,루돌프 니코,SOLO,r7qW6IsYR5pC4O_kty2UGC8DStHz30UpdvOKPI4rWu6IIp...,3.962264,0.444444
290,KR_6434537939,35.63,7,TOP,4,TwistedFate,TOP,202,True,0,8,9,xYlXPAcRCPRnTNzAuRmYYw-ZifNM2kCdCxr24Kg6910Ezd8,61,털잠옷,SOLO,IZRT6zROp4WS1-CKKeXitMytxMlyY1hRdCIbt5Tw-LCW2k...,5.669380,1.125000
336,KR_6434864284,29.55,6,TOP,107,Rengar,TOP,207,True,13,3,6,_HoWBAG3G_micLVVVxJ2rKYQD1yi8x-k_ORTC-MW-_HtfONi,34,고병억,SOLO,5cvGdDNJ9sdKtKOvErqQXs63We49Yp5vchu7UiLNohabBg...,7.005076,6.333333


# 2. 미드 라인에서 CS가 하위 15%인 경우

In [ ]:
def mid_fast(df):
    mid_df = df[df.lane.eq("MIDDLE") & df.individualPosition.eq("MIDDLE")]
    minionsKillperMin_bottom_15 = mid_df.minionsKillperMin.quantile(0.10)
    return mid_df[mid_df.minionsKillperMin < minionsKillperMin_bottom_15].sort_values('minionsKillperMin', ascending=True)

In [11]:
mid_match_df = match_df[match_df.lane.eq("MIDDLE") & match_df.individualPosition.eq("MIDDLE")]
minionsKillperMin_bottom_15 = mid_match_df.minionsKillperMin.quantile(0.10)
mid_match_df[mid_match_df.minionsKillperMin < minionsKillperMin_bottom_15].sort_values('minionsKillperMin', ascending=True)

,match_id,play_time,assistMePings,lane,championId,championName,individualPosition,totalMinionsKilled,win,kills,deaths,assists,summonerId,summonerLevel,summonerName,role,puuid,minionsKillperMin,kda
742,KR_6437148251,37.53,1,MIDDLE,12,Alistar,MIDDLE,81,True,0,11,8,7P0aGDWD8GLkxw1YxSMaapofOYmH0MsdMO7cs1vMiYbalf0,208,8시36분,CARRY,-fDyXXPCXC4IiowFu5hLuVkw81Ktz2DUgQpKUIeui82Fvj...,2.158273,0.727273
623,KR_6436652339,32.00,0,MIDDLE,76,Nidalee,MIDDLE,98,False,3,8,5,6IxsVBhyj04E3DuGUzsWrQJ35lPTPdUVQVG8iMtFb9mEBA,99,조매력조간지,SOLO,f-L0APf-A2JjmSaqPU9-fbRuy_K-A1xdqDMHCYq0ikradZ...,3.062500,1.000000
772,KR_6437185724,35.92,0,MIDDLE,7,Leblanc,MIDDLE,125,True,8,5,7,1h8oXQtHHfNqThcAbgs4UtpA5KL9rcutRQgaVO9resJ1wiI,160,20학번 혜지,SOLO,XCmJeNGRNZNqtNeCcCLsfeAhXw878kC5DJFkLG_de1gLNY...,3.479955,3.000000
675,KR_6436798432,34.70,0,MIDDLE,142,Zoe,MIDDLE,121,True,5,11,10,Br7JDWCiK63q0VgVNJ7kooN98NzR6MEgckbM0_v19O_mNXM,406,너미드미아,SOLO,xS7VlG-D7kcE9flo0krKL4LJD9aeNyqJ-om301atmgxBBc...,3.487032,1.363636
862,KR_6437387307,32.85,0,MIDDLE,68,Rumble,MIDDLE,118,True,7,5,12,KCMA70RdrqYSKZFHR8YHIf-yIr7HiFHs0h8t1xfnALIyLQ,444,매운맛 카리나,SOLO,b0Kn86EWmZpqWG4wbyb-W9aMsNhY6wYU7rEtQRSPcKPrXR...,3.592085,3.800000
402,KR_6435494168,28.02,0,MIDDLE,84,Akali,MIDDLE,101,False,3,6,7,pzSbCRPYWmGUeQISvSezAbszq5B5nq10kZEII5LowU6oT6U,276,싫으냥,SOLO,3K8_q9RwXKZ6bc7n38zgt0ly5A_HCJc5rX4RzLE43GgYZS...,3.604568,1.666667
27,KR_6432977223,27.28,0,MIDDLE,39,Irelia,MIDDLE,102,False,2,5,0,_FACJPTnLWpb1eiGdfybmP3GEK1-KPe39offWegUItQ6Q2s,285,In My Memory,SOLO,F0tNm_y4cPNAL4fJq6TyP4lNDBNunPa1wtWLeFEmhvf8ud...,3.739003,0.400000
900,KR_6437506086,28.52,1,MIDDLE,35,Shaco,MIDDLE,108,False,6,7,5,_O_-3haI88sZL4f3AKBjxMAXfj3G99V7bPij-1ZFUrdxLlw,95,남 택 호,SOLO,QsyBIaQL-RT0jb5LLJalZkmf0Ob8NvTpjPTl99X2uNKnr9...,3.786816,1.571429
547,KR_6436432607,37.83,3,MIDDLE,68,Rumble,MIDDLE,151,True,11,11,14,FVNm0x4VkBKTva94l1oh1RF-fKAxXS4pxAjpnRlmT2TyR48,412,럼킹블,SOLO,OBxjok_YaW-1CQMgegYbR0bOx9aUr5J3vVN-h3jrGtXS6F...,3.991541,2.272727
522,KR_6436404947,36.65,0,MIDDLE,246,Qiyana,MIDDLE,150,True,21,11,9,wRu4l3c4nk-bEwhJmoDl1XTKdGuf5zKi7PCNib87U0waMw,323,전혀문제없어,SUPPORT,BVTTLRD29xMhCJnU9YxpdF7v2nguAJH2GDhh0KNNBtjoyZ...,4.092769,2.727273


# 3. 정글 라인에서 킬과 어시스트가 낮으면서 분당 CS가 높은 경우

In [ ]:
def get_mmorpg_player(df):
    jungle_match_df = df[df.lane.eq("JUNGLE") & df.individualPosition.eq("JUNGLE")]
    jungle_top_cs = jungle_match_df.minionsKillperMin.describe()['75%']
    jungle_mean_kda = jungle_match_df.kda.mean()
    return jungle_match_df[(jungle_match_df.minionsKillperMin > jungle_top_cs) & (jungle_match_df.kda < jungle_mean_kda)].sort_values('minionsKillperMin', ascending=False)

In [13]:
jungle_match_df = match_df[match_df.lane.eq("JUNGLE") & match_df.individualPosition.eq("JUNGLE")]
jungle_top_cs = jungle_match_df.minionsKillperMin.describe()['75%']
jungle_mean_kda = jungle_match_df.kda.mean()
jungle_match_df[(jungle_match_df.minionsKillperMin > jungle_top_cs) & (jungle_match_df.kda < jungle_mean_kda)].sort_values('minionsKillperMin', ascending=False)

,match_id,play_time,assistMePings,lane,championId,championName,individualPosition,totalMinionsKilled,win,kills,deaths,assists,summonerId,summonerLevel,summonerName,role,puuid,minionsKillperMin,kda
896,KR_6437455524,23.05,0,JUNGLE,48,Trundle,JUNGLE,89,False,5,5,5,IMDXeXy-ONllCZ6spfqWckYF6rSxBLg2FBrnuR_ABGb_0qw,404,shtmdghks,NONE,b_KXfzmsltJ8vil-HkCfV42UcZ_ELMhe-8eDvlMQg7v5b7...,3.861171,2.000000
386,KR_6435402918,33.10,0,JUNGLE,245,Ekko,JUNGLE,96,False,16,7,5,jICqhL29fZiBpuuULtguN4DQgfJXSHp2dNo0DUVhlR0ODg,116,강형욱이버린불독,NONE,jptoD4x89a3gQWyAawCSk0txy7fqMgXHlXykqrkYArjmIM...,2.900302,3.000000
696,KR_6436853204,37.10,2,JUNGLE,91,Talon,JUNGLE,102,True,5,8,9,_SNggiaMAt4CuXZ6heM2FHHscvolk8nphlk8hRwuIFmXa0U,89,쓱을흙,NONE,PEGKNTQlebZqMQF-Jw0gDTxgJXT1o_0pRdwjVw6TZC_ztg...,2.749326,1.750000
441,KR_6435702020,33.90,3,JUNGLE,200,Belveth,JUNGLE,85,False,11,13,7,ZvJfXk5QIQCLbIlRRSuUy8eTsGFRAlkJmGomO0v-FClsEGM,92,PlXLD,NONE,DbHNQzIrX9YKrnOC6gJydeGn55OGW0EKCaVBWrOboGQFGE...,2.507375,1.384615
859,KR_6437380841,22.83,3,JUNGLE,518,Neeko,JUNGLE,55,False,5,11,11,-tkXG5PfpKgmANBeNLFhK-TvhxxuEJwQFuUQI0CBFTiyerA,383,주식은콩,NONE,82TjfVHQz_RiLNosfmI2IdAcX0vJ7flNQOHyeG7-8uA_-E...,2.409111,1.454545
291,KR_6434537939,35.63,0,JUNGLE,104,Graves,JUNGLE,74,True,6,5,12,_8JXNYxzSqXi5Gqr_oILGmFdNtAxoQGnU9hl2Bos4SirR6Q,99,Peca,NONE,EDQ4HPZ-2hLrVncIGy5QpgFu4wcoI6LwfIv480qx-ocZju...,2.076901,3.600000
921,KR_6437958352,40.80,0,JUNGLE,120,Hecarim,JUNGLE,83,False,7,9,20,jDM7kYYu27wXPDDHHU9wL8ssRl27O0ThfJlRw0_PVRej9Og,321,서울사는오상민,NONE,ed6qDSk12hJLQ_CCdJYTyerLnB3TdYQrZPiibWBm26tGFu...,2.034314,3.000000
701,KR_6436886927,33.62,2,JUNGLE,79,Gragas,JUNGLE,66,False,9,7,8,_SNggiaMAt4CuXZ6heM2FHHscvolk8nphlk8hRwuIFmXa0U,89,쓱을흙,NONE,PEGKNTQlebZqMQF-Jw0gDTxgJXT1o_0pRdwjVw6TZC_ztg...,1.963117,2.428571
326,KR_6434831957,37.97,1,JUNGLE,238,Zed,JUNGLE,71,True,11,7,14,zrqbr6-BqqTDksy6As2aMQLmx1M3WlfjVri8t_atCEfcpKE,203,유셩호,NONE,QkTkh8JuXv0y5puXqpJoq94Ok_rOdHUSVoB__1D1bCqYRE...,1.869897,3.571429
76,KR_6433165444,20.93,0,JUNGLE,245,Ekko,JUNGLE,38,False,5,7,6,f_0agU5j4G4PdIFF8J6MBDLhlvDfdH3CIQCiysLjmJW-AEU,424,둔 댱,NONE,LqIbXqMLL5ZOFIg6pumOERoOXK9p8t27wsT_-QZdMX2pbL...,1.815576,1.571429


# 4. 원딜 라인에서 CS가 하위 10%인 경우

In [ ]:
def bottom_fast(df):
    bottom_match_df = df[df.lane.eq("BOTTOM") & df.individualPosition.eq("BOTTOM") & df.role.ne("SUPPORT")]
    minionsKillperMin_bottom_10 = bottom_match_df.minionsKillperMin.quantile(0.10)
    return bottom_match_df[bottom_match_df.minionsKillperMin < minionsKillperMin_bottom_10].sort_values('minionsKillperMin', ascending=True)

In [16]:
bottom_match_df = match_df[match_df.lane.eq("BOTTOM") & match_df.individualPosition.eq("BOTTOM") & match_df.role.ne("SUPPORT")]
minionsKillperMin_bottom_10 = bottom_match_df.minionsKillperMin.quantile(0.10)
bottom_match_df[bottom_match_df.minionsKillperMin < minionsKillperMin_bottom_10].sort_values('minionsKillperMin', ascending=True)

,match_id,play_time,assistMePings,lane,championId,championName,individualPosition,totalMinionsKilled,win,kills,deaths,assists,summonerId,summonerLevel,summonerName,role,puuid,minionsKillperMin,kda
858,KR_6437380841,22.83,0,BOTTOM,67,Vayne,BOTTOM,50,False,4,4,8,K-MAWyFuBh6T7xfUAO8Kxxh61e6-MRY9Tj34eIdm2-0jOBA,247,도 구 해,DUO,eWY-eTjySERrThQ1zT2TsUoi1UrRl4ZMWjIFRnAaTXRpZX...,2.190101,3.000000
313,KR_6434788812,31.47,0,BOTTOM,81,Ezreal,BOTTOM,112,False,2,12,8,RhvnLnG-dbrcvNc3p_CCqXBbon7T1Vw9E9tI2uJQvCc59RA,484,민형이서폿갈래,CARRY,njD2oZvMOolkrGkcylceKh-5z3baNCeWvpK_VeGhP5LBSg...,3.558945,0.833333
944,KR_6438040750,26.58,1,BOTTOM,498,Xayah,BOTTOM,96,False,2,15,7,G8n7NqqVbzTgw3yT_GDSfPrzA2DN9-JBvMfyCUkboSTtgt...,162,헛개수빠차,CARRY,1H6cs4obm-mzBipcaW23IdAsg4cgHLHUWOeE5_7jm9Njd-...,3.611738,0.600000
168,KR_6434066105,40.05,0,BOTTOM,31,Chogath,BOTTOM,166,True,14,10,12,7xeIj8tZWcej-nMkfJd_woa_aLMrDOEiOfNs0kX_j0G54Q,353,해결방안,CARRY,joawlwTz2YrICzFoTRYTjeOA4cixNUyyU5MnTHTAhincwG...,4.144819,2.600000
968,KR_6438071184,30.40,0,BOTTOM,360,Samira,BOTTOM,131,True,6,7,5,PfM8oVT35bz72fE2hWF8a7dJhJidxoZN80WELJvl5TjPe_w,455,겜중이면대리임,CARRY,hCbfjRQhJxO-sUuVFUgL4eAbeibRRV0VDhWLtt1eMNXE-l...,4.309211,1.571429
868,KR_6437387307,32.85,4,BOTTOM,119,Draven,BOTTOM,143,False,15,12,2,27Pj97M_8ACm97bs2vfrmFkzGrAJhPCl-1uVRY5Y4g54hU...,61,ellim 303,DUO,KFeeyS9Sqq8A20YNgUedJlm9urN1LaeGzkZfyAhB4jpdcq...,4.353120,1.416667
857,KR_6437380841,22.83,0,BOTTOM,20,Nunu,BOTTOM,102,False,0,3,8,UGcY4ETpmSUEHP3GfIK4879H-xPrsDpLAK0B3lQMvDU-tIs,184,천안 임윤수,DUO,8sl42_eArtFA_VU7P3ROhx0zz2VAJbUrGnJOF9OP4j2uWq...,4.467806,2.666667
888,KR_6437444515,27.70,1,BOTTOM,145,Kaisa,BOTTOM,127,False,2,13,5,2GLNskfnxY0p57Lc7cR_e87dEKIZY_R4v6YIQNsyBsmMZkk,108,마시마로7,CARRY,Ymoj-GmiJSzAm7e0ts4Rd01XNzbTEUin0BIxeBxc2LXIaf...,4.584838,0.538462
808,KR_6437229143,26.23,0,BOTTOM,35,Shaco,BOTTOM,121,False,2,3,5,UGxUnWYOiKK8YosVJL9kvJARKT6q3UdoB9bJKyL5ppINL8A,246,앞무빙치는원딜,CARRY,fxr7UqmGyI-vfFn1hTCrXBmuozA2aBlDLZuj9mamv3GT-N...,4.613039,2.333333
973,KR_6438121442,33.27,0,BOTTOM,67,Vayne,BOTTOM,154,True,25,5,14,Jxmd6LTDe53h5t5skSjhOyV-fz2Xx7h60LmmRsQARGd_nA,106,멜랑골리,CARRY,AqBETj7th4D3a0ZbIEvt3XNSL7WvjGcXHsQRzMwfe9qwfj...,4.628795,7.800000


# 5. 서포터 라인인데 CS가 원딜 평균보다 높음 + 평균 서포터 CS보다 높은 경우

In [ ]:
def support_cs_thief(df):
    support_match_df = df[df.lane.eq("BOTTOM") & df.individualPosition.eq("UTILITY") & df.role.eq("SUPPORT")]
    mean_support_minion_kill_per_min = support_match_df.minionsKillperMin.describe()['75%']
    return support_match_df[support_match_df.minionsKillperMin > mean_support_minion_kill_per_min].sort_values('minionsKillperMin', ascending=False)

In [17]:
support_match_df = match_df[match_df.lane.eq("BOTTOM") & match_df.individualPosition.eq("UTILITY") & match_df.role.eq("SUPPORT")]
mean_support_minion_kill_per_min = support_match_df.minionsKillperMin.describe()['75%']
support_match_df[support_match_df.minionsKillperMin > mean_support_minion_kill_per_min].sort_values('minionsKillperMin', ascending=False)


,match_id,play_time,assistMePings,lane,championId,championName,individualPosition,totalMinionsKilled,win,kills,deaths,assists,summonerId,summonerLevel,summonerName,role,puuid,minionsKillperMin,kda
89,KR_6433212646,35.05,12,BOTTOM,9,FiddleSticks,UTILITY,100,False,4,9,5,VBD3LQ4l7JC0oReSJoyxORBxjkLdF0sPUZJ7IugjNeL8gA,388,솔킬시안감,SUPPORT,MaUKBMZbVyWjJUCoNf9BmhVjUpA-7Bxl89hTBCPm_5Lw4z...,2.853067,1.000000
729,KR_6437086877,30.88,0,BOTTOM,3,Galio,UTILITY,79,False,3,4,16,9zKDHzr2Vr970guEPL0aBpDI-Y8YrdKhJVsmyr4tWE_wgsQ,640,Craftsman 1127,SUPPORT,4dVhfCpbp22ur-odG353i4Ns4DgPsdnswVppVQ3jy5QswE...,2.558290,4.750000
609,KR_6436592141,32.15,3,BOTTOM,147,Seraphine,UTILITY,80,True,3,6,13,dKzSNp751GMvLt1W22PKuRFIXDxojGFAUTbKKo01tlZnZ1M,74,공주는꽃이좋아,SUPPORT,vjMeLH_jN-N6KqYEdS9hCyHrqHoFEvbxPxI66W45CG7Ezv...,2.488336,2.666667
164,KR_6434066105,40.05,0,BOTTOM,143,Zyra,UTILITY,99,False,6,12,22,l0_EUoxCbx2gYuIUSzvosYTt1fGLAg4BqMgARysQ8UDvgQ,239,그녀의 손기술,SUPPORT,2RZHiqP8GEtybNDxoPOzT1ArBrfmdfKKOQIolTd080bMJB...,2.471910,2.333333
499,KR_6436294536,29.02,0,BOTTOM,99,Lux,UTILITY,71,False,4,4,9,9axLh1J-8FiskGpwdJ-g_B7p1ekWcIYpHuNAd95TRARImA,339,용우용,SUPPORT,UsGfC0v5ThNUJVwvDKNkRmUwf4LvzY-T_TXSQKUdrtFoUw...,2.446589,3.250000
299,KR_6434537939,35.63,1,BOTTOM,45,Veigar,UTILITY,86,False,6,5,7,6NyDZ4dQ8ZzyP-9p8Sp3lGElOzEoHBWoG0AiJu9FUNZdjIk,74,흔들리는버스,SUPPORT,KkFjRqUO5e8BqlAS_e698ZQUHSOfconWG3Lmi0RUE1l0RW...,2.413696,2.600000
824,KR_6437301428,30.22,8,BOTTOM,122,Darius,UTILITY,71,False,3,3,12,niRxZXz5NSV8TLhsL2vNWcjbymrh1UrJcABcCF5SsXw7GeI,252,노네띠,SUPPORT,1wtj9cUqpit7MefCmx-kx0U-73-7eYtkjndzZuwQMs78TA...,2.349437,5.000000
39,KR_6432988425,24.15,3,BOTTOM,154,Zac,UTILITY,52,True,2,2,14,WtOTZpnECie1-5NVH5d42qZAhB-Aya9b9DiFJ-dTlkiWnB8,479,도무탄,SUPPORT,0zwEA5FDYOgd3ca_irBMwUhgcvcX4lyPS4iODXSfjW_vfl...,2.153209,8.000000
519,KR_6436359738,30.88,0,BOTTOM,875,Sett,UTILITY,66,False,4,9,5,u69VaobR7ZTLndUP7-cZQ_7LeTZhqVTKFryaMh3Q_IzFr8s,523,알파카탄임금,SUPPORT,zvDP-bIy2x95gPFMFUFv6C-1cTzQ8NqFUD1PN8U8okMLvW...,2.137306,1.000000
424,KR_6435601122,33.65,2,BOTTOM,99,Lux,UTILITY,71,False,7,8,15,9zsLxWdZPuQsaixs2-r_HDSHEcdzT4pWE1gqazvdGEtspJk,197,이강택,SUPPORT,dxIbuNtXwYD2h5cUw78aLid7husIUs0NWqMxm1mLga5i8B...,2.109955,2.750000


In [13]:
match_df.individualPosition.unique()

array(['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY'], dtype=object)

In [14]:
match_df.role.unique()

array(['NONE', 'SOLO', 'CARRY', 'SUPPORT', 'DUO'], dtype=object)

In [15]:
match_df.lane.unique()

array(['JUNGLE', 'MIDDLE', 'BOTTOM', 'TOP', 'NONE'], dtype=object)

In [17]:
match_df[match_df.lane.eq("NONE")]

,match_id,play_time,assistMePings,lane,championId,championName,individualPosition,totalMinionsKilled,win,kills,deaths,assists,summonerId,summonerLevel,summonerName,role,puuid,minionsKillperMin,kda
10,KR_6432772335,18.85,0,NONE,122,Darius,TOP,126,True,5,1,1,FBljcgwuTEQnumLG1Og03OwZrn-rw9I5KaSgX58go2N2mZ...,58,내가 실수했어,SUPPORT,VFdJ6pXCQZR-1pc5QRMTN4zGpn2gYxJAcNdgClHZZP9UMT...,6.684350,6.000000
11,KR_6432772335,18.85,0,NONE,163,Taliyah,JUNGLE,17,True,9,1,3,GnnqgCezKUr4DlyM3bVBrZ2sjR7jz0yAM1s1lFbG6G-vHQ,187,아 저,SUPPORT,_3622zNTPmzRI1Xr8gbB_X93X0WWntMm9sc2AiloWzYKzK...,0.901857,12.000000
12,KR_6432772335,18.85,3,NONE,711,Vex,MIDDLE,133,True,3,1,2,yvAr6acNxHjuUE1NBHhN5h2fjhuK1JONr3kuT5bbr3fSHg,230,섹시복근,SUPPORT,Vnp4CqtVTI1p5Z76yPNgT-wffxGqwQjU7YyCCoUe8mepFK...,7.055703,5.000000
13,KR_6432772335,18.85,5,NONE,498,Xayah,BOTTOM,155,True,4,0,5,GTwp8WdE0dOEDxZ-SvBhK7OYDRJ98DRZOwfGnK2gWAS1yA,516,진아가라사데,SUPPORT,FeHiH68wlpxy-kVb_7CSK3_xWecqt8oo-IIW0xFx3mNtgT...,8.222812,inf
14,KR_6432772335,18.85,0,NONE,497,Rakan,UTILITY,28,True,1,2,7,cl9hg947Y9LRm5fcKJoZ0YM8yEPhyCXXnYbsky9m4kvhTw,223,동물농장두더지,SUPPORT,TriAHADv4i7nB6aezlcnFf-7TWrKUH1CdKPIeIezs9QmH8...,1.485411,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,KR_6437413398,19.58,0,NONE,875,Sett,TOP,111,False,2,3,2,tnl2tfRtuIA4d7Gs9IVXpqhi1GIuR8h8lyutmtknx3g_I_g,293,Zhongtu paobu,SUPPORT,7FRCwO3Tn4Sv33P6sSRwWFlBI3WerqmvgQBQfTirjlRpkl...,5.669050,1.333333
876,KR_6437413398,19.58,1,NONE,141,Kayn,JUNGLE,6,False,3,7,1,_5_FCaxapBoWQT4CKAkAGwkvlwt5aIomo3WqGWsqyOQuYYg,279,브래드톡,SUPPORT,j0P5jYnisjzzfS-fsCpepqVuApsv9YM82NoB64poucGK8I...,0.306435,0.571429
877,KR_6437413398,19.58,0,NONE,112,Viktor,MIDDLE,86,False,1,10,0,bsIlC5zCgFhQ-GsEa3Q_q9m6fO4-7V0UnnVZALTU5YLVBlE,175,enpri,SUPPORT,yh2jB-Q46NyG1CYFypLIg5hXtQzaECalAzBRnR1rTDPU8x...,4.392237,0.100000
878,KR_6437413398,19.58,0,NONE,202,Jhin,BOTTOM,113,False,1,5,2,DbJzfDTi4aU3Ihlo75Y_rqnONUnFBbElwQdod17pL2LrVNQJ,113,쿄와이네,DUO,yb_RizjpYT0xSy0l9rmPFM1-okUPPqnPHCWmVxbvdbT3UF...,5.771195,0.600000
